# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
cityData = pd.read_csv("../output_data/cities.csv")

# Display sample data
cityData.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dudinka,69.4058,86.1778,-7.31,95,90,1.58,RU,1698088937
1,1,ust-nera,64.5667,143.2000,-14.55,95,90,2.56,RU,1698088937
2,2,albany,42.6001,-73.9662,14.46,56,10,0.89,US,1698088937
3,3,quellon,-43.1167,-73.6167,13.43,69,9,4.06,CL,1698088937
4,4,san patricio,28.0170,-97.5169,30.97,52,46,9.24,US,1698088938


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
mapPlot = cityData.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
mapPlot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
#Something is wrong in this cell, which results in output given varying from the given by a large margin
maxTemp = 27
minTemp = 21
tempInclusive = True
idealWind = 4.5
idealCloud = 0

narrowDF = cityData.loc[(cityData['Max Temp'].between(minTemp, maxTemp)) & (cityData['Wind Speed'] < idealWind) & (cityData['Cloudiness'] == idealCloud)]

# Drop any rows with null values
narrowDF = narrowDF.dropna()

# Display sample data
narrowDF

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,dickson,36.0770,-87.3878,26.09,33,0,2.57,US,1698088801
58,58,siwa oasis,29.2041,25.5195,25.29,34,0,2.20,EG,1698088945
70,70,port hedland,-20.3167,118.5667,23.03,94,0,3.60,AU,1698088759
141,141,inhambane,-23.8650,35.3833,21.50,78,0,1.97,MZ,1698088958
245,245,saint-pierre,-21.3393,55.4781,23.82,73,0,1.54,RE,1698088972
306,306,liwale,-9.7667,37.9333,24.59,57,0,2.98,TZ,1698088981
309,309,pahrump,36.2083,-115.9839,21.94,39,0,3.58,US,1698088982
349,349,letlhakeng,-24.0944,25.0298,24.25,13,0,2.82,BW,1698088987
361,361,mehndawal,26.9753,83.1117,21.68,65,0,1.58,IN,1698088989
428,428,ermoupolis,37.4423,24.9425,21.76,83,0,0.00,GR,1698089020


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotelDF = narrowDF.copy()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotelDF['Hotel Name'] = ""

# Display sample data
hotelDF

,City,Country,Lat,Lng,Humidity,Hotel Name
54,dickson,US,36.0770,-87.3878,33,
58,siwa oasis,EG,29.2041,25.5195,34,
70,port hedland,AU,-20.3167,118.5667,94,
141,inhambane,MZ,-23.8650,35.3833,78,
245,saint-pierre,RE,-21.3393,55.4781,73,
306,liwale,TZ,-9.7667,37.9333,57,
309,pahrump,US,36.2083,-115.9839,39,
349,letlhakeng,BW,-24.0944,25.0298,13,
361,mehndawal,IN,26.9753,83.1117,65,
428,ermoupolis,GR,37.4423,24.9425,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories' : 'accommodation.hotel',
    'apiKey' : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotelDF.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotelDF.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotelDF.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotelDF.loc[index, 'City']} - nearest hotel: {hotelDF.loc[index, 'Hotel Name']}")

# Display sample data
hotelDF

Starting hotel search
dickson - nearest hotel: No hotel found
siwa oasis - nearest hotel: فندق الكيلانى
port hedland - nearest hotel: The Esplanade Hotel
inhambane - nearest hotel: Hotel de Inhambane
saint-pierre - nearest hotel: Tropic Hotel
liwale - nearest hotel: No hotel found
pahrump - nearest hotel: Holiday Inn Express & Suites
letlhakeng - nearest hotel: No hotel found
mehndawal - nearest hotel: No hotel found
ermoupolis - nearest hotel: Παλλάδιον
big bend - nearest hotel: No hotel found
canon city - nearest hotel: No hotel found
mersa matruh - nearest hotel: صقور


,City,Country,Lat,Lng,Humidity,Hotel Name
54,dickson,US,36.0770,-87.3878,33,No hotel found
58,siwa oasis,EG,29.2041,25.5195,34,فندق الكيلانى
70,port hedland,AU,-20.3167,118.5667,94,The Esplanade Hotel
141,inhambane,MZ,-23.8650,35.3833,78,Hotel de Inhambane
245,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
306,liwale,TZ,-9.7667,37.9333,57,No hotel found
309,pahrump,US,36.2083,-115.9839,39,Holiday Inn Express & Suites
349,letlhakeng,BW,-24.0944,25.0298,13,No hotel found
361,mehndawal,IN,26.9753,83.1117,65,No hotel found
428,ermoupolis,GR,37.4423,24.9425,83,Παλλάδιον


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
mapPlot = hotelDF.hvplot.points(
    'Lng',
    'Lat',
    hover_cols = ['City', 'Hotel Name'],
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
)

# Display the map
mapPlot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)